# Exercício: Análise de Importância na Wikipedia com PageRank

## Contexto

Você é um pesquisador analisando a estrutura de conhecimento da Wikipedia usando o dataset WikiSpeedia. Esta rede representa artigos interconectados através de hyperlinks.Seu objetivo é identificar os artigos mais "importantes" usando o algoritmo PageRank e comparar com outras medidas de centralidade.

## Sobre as Categorias do Dataset

O dataset WikiSpeedia inclui um sistema de categorização hierárquica dos artigos. Cada artigo pode pertencer a uma ou mais categorias, organizadas em níveis (ex: "Geography.Countries.United_States"). Para agrupar por área de conhecimento, você deverá:

1. Utilizar o arquivo categories.tsv que mapeia cada artigo para suas categorias

2. Definir áreas de conhecimento amplas agrupando categorias específicas

3. Criar um mapeamento entre categorias detalhadas e áreas principais

### Áreas de Conhecimento Sugeridas:

- Ciências: Mathematics, Physics, Chemistry, Biology, Astronomy, Science

- Geografia: Geography, Countries, Cities, Rivers, Mountains, Places

- História: History, Ancient_history, Medieval_history, War, Historical_events

- Arte e Cultura: Art, Music, Literature, Film, Culture, Entertainment

- Tecnologia: Technology, Computing, Internet, Engineering, Invention

- Sociedade: Society, Politics, Law, Education, Philosophy


In [ ]:
!wget https://snap.stanford.edu/data/wikispeedia/wikispeedia_paths-and-graph.tar.gz

--2025-10-02 22:39:07--  https://snap.stanford.edu/data/wikispeedia/wikispeedia_paths-and-graph.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9901821 (9.4M) [application/x-gzip]
Saving to: ‘wikispeedia_paths-and-graph.tar.gz’

wikispeedia_paths-a 100%[===================>]   9.44M  1.82MB/s    in 5.2s    

2025-10-02 22:39:13 (1.82 MB/s) - ‘wikispeedia_paths-and-graph.tar.gz’ saved [9901821/9901821]



In [ ]:
!tar xzf ./wikispeedia_paths-and-graph.tar.gz

In [ ]:
import pandas as pd

# Utilizar o arquivo categories.tsv que mapeia cada artigo para suas categorias
categorias = pd.read_csv('./wikispeedia_paths-and-graph/categories.tsv', sep='\t', header=None, names=['article', 'category'])

# Definir áreas de conhecimento amplas agrupando categorias específicas
areas_de_conhecimento = {
    'Ciências': ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Astronomy', 'Science'],
    'Geografia': ['Geography', 'Countries', 'Cities', 'Rivers', 'Mountains', 'Places'],
    'História': ['History', 'Ancient_history', 'Medieval_history', 'War', 'Historical_events'],
    'Arte e Cultura': ['Art', 'Music', 'Literature', 'Film', 'Culture', 'Entertainment'],
    'Tecnologia': ['Technology', 'Computing', 'Internet', 'Engineering', 'Invention'],
    'Sociedade': ['Society', 'Politics', 'Law', 'Education', 'Philosophy']
}

# Criar um mapeamento entre categorias detalhadas e áreas principais
def mapear_area(categoria):
    if pd.isna(categoria):
        return None
    for area, cats in areas_de_conhecimento.items():
        for c in cats:
            if c.lower() in categoria.lower():
                return area
    return None

# Adicionar coluna com a area geral correspondente
categorias['area_principal'] = categorias['category'].apply(mapear_area)


In [ ]:
import networkx as nx

# Carregar ligacoes entre artigos
links = pd.read_csv('./wikispeedia_paths-and-graph/links.tsv', sep='\t', header=None, names=['source', 'target'])

G = nx.DiGraph()
for _, row in links.iterrows():
    G.add_edge(row['source'], row['target'])

pagerank = nx.pagerank(G)

# Juntar PageRank ao dataframe de categorias
categorias['pagerank'] = categorias['article'].map(pagerank)

pagerank_area = categorias.groupby('area_principal')['pagerank'].mean().sort_values(ascending=False)
print(pagerank_area)


area_principal
Geografia         0.000448
Sociedade         0.000341
História          0.000203
Ciências          0.000177
Arte e Cultura    0.000170
Tecnologia        0.000107
Name: pagerank, dtype: float64


---

## Parte 1: Análise Básica do PageRank

**Questão 1**: Após calcular o PageRank para todos os artigos, quais são os 10 artigos com maior score? Liste-os em ordem decrescente de importância.

**Questão 2**: Analisando os resultados do top 10, que áreas de conhecimento parecem
dominar? Por que você acha que isso acontece?

**Questão 3**: Teste o algoritmo com diferentes fatores de amortecimento (α = 0,5, 0,85 e 0,95). Como mudam os rankings? Explique o efeito deste parâmetro.

In [ ]:
# Questão 1: Após calcular o PageRank para todos os artigos, quais são os 10 artigos com maior score? Liste-os em ordem decrescente de importância.
top_10_artigos = categorias.nlargest(10, 'pagerank')
print(top_10_artigos[['article', 'pagerank']])

               article  pagerank
4860     United_States  0.009559
4861     United_States  0.009559
1784            France  0.006418
1785            France  0.006418
1628            Europe  0.006335
4854    United_Kingdom  0.006230
4855    United_Kingdom  0.006230
4856    United_Kingdom  0.006230
1576  English_language  0.004861
1929           Germany  0.004821


Questão 2: Analisando os resultados do top 10, que áreas de conhecimento parecem dominar? Por que você acha que isso acontece?

No Top 10, a area Geografia domina porque paises e linguas sao nós altamente conectados, funcionando como atalhos universais dentro da rede de artigos.

In [ ]:
# Questão 3: Teste o algoritmo com diferentes fatores de amortecimento (α = 0,5, 0,85 e 0,95). Como mudam os rankings? Explique o efeito deste parâmetro.
for alpha in [0.5, 0.85, 0.95]:
    pagerank = nx.pagerank(G, alpha=alpha)
    categorias['pagerank_alpha_' + str(alpha)] = categorias['article'].map(pagerank)
    top_10_artigos_alpha = categorias.nlargest(10, 'pagerank_alpha_' + str(alpha))
    print("\n\nTop 10 artigos com alpha = {}:".format(alpha))
    print(top_10_artigos_alpha[['article', 'pagerank_alpha_' + str(alpha)]])




Top 10 artigos com alpha = 0.5:
             article  pagerank_alpha_0.5
4860   United_States            0.006918
4861   United_States            0.006918
4854  United_Kingdom            0.004080
4855  United_Kingdom            0.004080
4856  United_Kingdom            0.004080
1628          Europe            0.003943
1784          France            0.003861
1785          France            0.003861
1572         England            0.003327
5145    World_War_II            0.003056


Top 10 artigos com alpha = 0.85:
               article  pagerank_alpha_0.85
4860     United_States             0.009559
4861     United_States             0.009559
1784            France             0.006418
1785            France             0.006418
1628            Europe             0.006335
4854    United_Kingdom             0.006230
4855    United_Kingdom             0.006230
4856    United_Kingdom             0.006230
1576  English_language             0.004861
1929           Germany             0.004

Questão 3: Teste o algoritmo com diferentes fatores de amortecimento (α = 0,5, 0,85 e 0,95). Como mudam os rankings? Explique o efeito deste parâmetro.

Com α baixo, o ranking fica mais democratico e aparecem artigos relevantes mas menos conectados. Já com α alto, o ranking fica mais elitista, privilegiando apenas os hubs com muitas conexoes. Por fim, o α = 0.85 costuma ser o melhor compromisso entre diversidade e representatividade estrutural.

---

Parte 2: Comparação com Outras Métricas de Centralidade

Questão 4: Calcule a centralidade de grau de entrada (in-degree) para todos os artigos. Quais são os 10 artigos com maior grau de entrada?

Questão 5: Compare as listas do top 10 do PageRank e do top 10 do grau de entrada. Quais artigos aparecem em ambas? Quais aparecem apenas em uma das listas?

Questão 6: Para os artigos que aparecem no topo do PageRank mas não no topo do grau de entrada (ou vice-versa), qual é a possível explicação? O que cada métrica está capturando de diferente?

In [ ]:
# Questão 4: Calcule a centralidade de grau de entrada (in-degree) para todos os artigos. Quais são os 10 artigos com maior grau de entrada?

in_degree = dict(G.in_degree())
categorias['in_degree'] = categorias['article'].map(in_degree)
top_10_artigos_in_degree = categorias.nlargest(10, 'in_degree')
print("Top 10 artigos com maior grau de entrada:")
print(top_10_artigos_in_degree[['article', 'in_degree']])

Top 10 artigos com maior grau de entrada:
             article  in_degree
4860   United_States     1551.0
4861   United_States     1551.0
4854  United_Kingdom      972.0
4855  United_Kingdom      972.0
4856  United_Kingdom      972.0
1784          France      959.0
1785          France      959.0
1628          Europe      933.0
1572         England      751.0
5145    World_War_II      751.0


Questão 5: Compare as listas do top 10 do PageRank e do top 10 do grau de entrada. Quais artigos aparecem em ambas? Quais aparecem apenas em uma das listas?

No top 10 de PageRank e grau de entrada aparecem United_States, United_Kingdom, France e Europe. English_language e Germany aparecem só no PageRank, England e World_War_II só no grau de entrada. O grau de entrada mede popularidade, o PageRank mede prestígio.

Questão 6: Para os artigos que aparecem no topo do PageRank mas não no topo do grau de entrada (ou vice-versa), qual é a possível explicação? O que cada métrica está capturando de diferente?

Artigos no topo do PageRank mas não no grau de entrada, como English_language e Germany, recebem links de artigos importantes, aumentando seu prestígio mesmo com menos links. Já artigos com alto grau de entrada mas menor PageRank, como England e World_War_II, recebem muitos links de artigos menos relevantes. Ou seja, o grau de entrada mede popularidade bruta, enquanto o PageRank mede prestígio ponderado pela importância de quem linka.

---

Parte 3: Análise por Áreas de Conhecimento

Questão 7: Agrupe os artigos por área de conhecimento (usando o mapeamento definido) e calcule o PageRank médio para cada área. Quais áreas têm os artigos mais "importantes" na rede?

Questão 8: Escolha três áreas diferentes e compare um artigo de alta importância de cada uma. O que esses artigos têm em comum em termos de sua posição na rede?

Questão 9: Existem áreas de conhecimento cujos artigos têm consistentemente baixo PageRank? O que isso pode indicar sobre a estrutura da rede?

In [ ]:
# Questão 7: Agrupe os artigos por área de conhecimento (usando o mapeamento definido) e calcule o PageRank médio para cada área.
# Quais áreas têm os artigos mais "importantes" na rede?

pagerank_area = categorias.groupby('area_principal')['pagerank'].mean().sort_values(ascending=False)
print("Artigos mais importantes por área:")
print(pagerank_area)

Artigos mais importantes por área:
area_principal
Geografia         0.000448
Sociedade         0.000341
História          0.000203
Ciências          0.000177
Arte e Cultura    0.000170
Tecnologia        0.000107
Name: pagerank, dtype: float64


Os artigos da área Geografia têm o maior PageRank médio, seguidos por Sociedade e História. Isso indica que, na rede, artigos sobre países, continentes e tópicos sociais são mais centrais e influentes, funcionando como hubs que conectam diversos outros artigos. As áreas Ciências, Arte e Cultura e Tecnologia aparecem com menor PageRank médio, mostrando que seus artigos são menos centrais na estrutura global da rede.


In [ ]:
# Questão 8: Escolha três áreas diferentes e compare um artigo de alta importância de cada uma.
# O que esses artigos têm em comum em termos de sua posição na rede?

areas_escolhidas = ['Ciências', 'Arte e Cultura', 'Tecnologia']
for area in areas_escolhidas:
    artigos_area = categorias[categorias['area_principal'] == area]
    artigo_mais_importante = artigos_area.nlargest(1, 'pagerank')
    print("\n\nArtigos mais importantes na área {}:".format(area))
    print(artigo_mais_importante[['article', 'pagerank']])



Artigos mais importantes na área Ciências:
        article  pagerank
4687  Time_zone  0.003463


Artigos mais importantes na área Arte e Cultura:
               article  pagerank
1576  English_language  0.004861


Artigos mais importantes na área Tecnologia:
         article  pagerank
4541  Television  0.001074


Todos os três artigos são hubs centrais, recebendo links de muitos outros artigos e conectando tópicos diversos, o que aumenta seu PageRank e os torna influentes na rede, independentemente da área.

In [ ]:
# Questão 9: Existem áreas de conhecimento cujos artigos têm consistentemente baixo PageRank?
# O que isso pode indicar sobre a estrutura da rede?
pagerank_area = categorias.groupby('area_principal')['pagerank'].mean().sort_values()
print(pagerank_area)

area_principal
Tecnologia        0.000107
Arte e Cultura    0.000170
Ciências          0.000177
História          0.000203
Sociedade         0.000341
Geografia         0.000448
Name: pagerank, dtype: float64


As áreas Tecnologia, Arte e Cultura e Ciências têm PageRank médio mais baixo, indicando que seus artigos são menos centrais na rede. Isso sugere que eles recebem menos links ou links de artigos menos influentes, funcionando como tópicos periféricos, enquanto Geografia e Sociedade têm artigos mais centrais e atuam como hubs de referência.

---

Parte 4: Casos de Estudo Específicos

Questão 10: Selecione um artigo do top 5 do PageRank e analise sua vizinhança na rede. Ele recebe links de que tipos de artigos? Para onde ele aponta? Isso explica sua alta importância?

Questão 11: Encontre um artigo que tenha alto grau de saída (muitos links para outros artigos) mas PageRank médio ou baixo. O que isso nos diz sobre como o PageRank funciona?

Questão 12: Identifique um artigo que seja "ponte" entre diferentes áreas de conhecimento.Como sua posição na rede se reflete em seu PageRank?

In [ ]:
# Questão 10: Selecione um artigo do top 5 do PageRank e analise sua vizinhança na rede.
# Ele recebe links de que tipos de artigos? Para onde ele aponta? Isso explica sua alta importância?
top_5_artigos = categorias.nlargest(5, 'pagerank')
for index, row in top_5_artigos.iterrows():
    article = row['article']
    neighbors = list(G.neighbors(article))
    print("\n\nVizinhos de {}:".format(article))
    print(neighbors)



Vizinhos de United_States:
['Abraham_Lincoln', 'Advertising', 'Agriculture', 'American_Civil_War', 'American_English', 'American_Revolutionary_War', 'American_Samoa', 'American_football', 'American_popular_music', 'Amtrak', 'Anguilla', 'Antarctica', 'Antigua_and_Barbuda', 'Apollo_11', 'Arctic_Ocean', 'Argentina', 'Aruba', 'Atlanta%2C_Georgia', 'Atlantic_Ocean', 'Attack_on_Pearl_Harbor', 'Auto_racing', 'Bah%C3%A1%27%C3%AD_Faith', 'Baker_Island', 'Bald_Eagle', 'Baltimore%2C_Maryland', 'Barbados', 'Baseball', 'Basketball', 'Battle_of_Gettysburg', 'Belize', 'Benin', 'Bermuda', 'Boston%2C_Massachusetts', 'Brazil', 'British_Virgin_Islands', 'Broadcasting', 'Brunei', 'Buddhism', 'C%C3%B4te_d%27Ivoire', 'California', 'Cambodia', 'Cameroon', 'Canada', 'Cape_Verde', 'Capitalism', 'Caribbean_Sea', 'Cayman_Islands', 'Chicago', 'China', 'Chinese_language', 'Christopher_Columbus', 'Cinema_of_the_United_States', 'Coal', 'Cold_War', 'Colombia', 'Communism', 'Computer', 'Corporation', 'Costa_Rica', '

United_States tem alta importância no PageRank porque recebe links de artigos de história, geografia, cultura e política, e também aponta para muitos desses temas, funcionando como hub central na rede.

In [ ]:
# Questão 11: Encontre um artigo que tenha alto grau de saída (muitos links para outros artigos) mas PageRank médio ou baixo.
# O que isso nos diz sobre como o PageRank funciona?
in_degree = dict(G.in_degree())
out_degree = dict(G.out_degree())
categorias['in_degree'] = categorias['article'].map(in_degree)
categorias['out_degree'] = categorias['article'].map(out_degree)
artigos_alto_grau_saida = categorias[categorias['out_degree'] > 100]
artigos_alto_grau_saida = artigos_alto_grau_saida.sort_values(by='pagerank', ascending=False)
print(artigos_alto_grau_saida[['article', 'pagerank', 'in_degree', 'out_degree']])

                                 article  pagerank  in_degree  out_degree
4861                       United_States  0.009559     1551.0       294.0
4860                       United_States  0.009559     1551.0       294.0
1628                              Europe  0.006335      933.0       159.0
4854                      United_Kingdom  0.006230      972.0       168.0
4855                      United_Kingdom  0.006230      972.0       168.0
...                                  ...       ...        ...         ...
2237             History_of_South_Africa  0.000064        5.0       105.0
1465                   Economy_of_Africa  0.000055        3.0       101.0
3622  Periodic_table_%28large_version%29  0.000037        1.0       118.0
1412        Driving_on_the_left_or_right  0.000033        0.0       255.0
2848                       List_of_lakes  0.000033        0.0       109.0

[128 rows x 4 columns]


Alguns artigos, como “Driving_on_the_left_or_right” e “List_of_lakes”, têm alto grau de saída (mais de 100 links apontando para outros artigos), mas recebem poucos ou nenhum link de entrada, resultando em um PageRank baixo ou médio.

Isso mostra que o PageRank mede a importância de um artigo com base nos links que ele recebe, e não na quantidade de links que ele faz. Um artigo pode apontar para muitos outros, mas se não for referenciado por artigos relevantes, seu PageRank permanece baixo.

In [ ]:
# Questão 12: Identifique um artigo que seja "ponte" entre diferentes áreas de conhecimento.
# Como sua posição na rede se reflete em seu PageRank?
artigos_ponte = categorias[categorias['area_principal'].isnull()]
artigos_ponte = artigos_ponte.sort_values(by='pagerank', ascending=False)
print(artigos_ponte[['article', 'pagerank']])

                                                article  pagerank
1253                                           Currency  0.003235
1032                                       Christianity  0.003016
2468                                              Islam  0.002713
3981                              Roman_Catholic_Church  0.002008
1762                              Football_%28soccer%29  0.001787
...                                                 ...       ...
0            # Hierarchical categories of all articles.       NaN
1     # Many articles have more than one category. S...       NaN
3                        # FORMAT:   article   category       NaN
500                                              Badugi       NaN
2886                          Lone_Wolf_%28gamebooks%29       NaN

[1171 rows x 2 columns]


Um exemplo é o artigo “Currency”, que conecta temas de Geografia, Economia e Sociedade. Ele atua como ponte entre diferentes áreas de conhecimento, recebendo links de artigos de várias categorias e apontando para muitos outros.

Sua posição na rede, ligando diferentes clusters, faz com que ele tenha um PageRank relativamente alto, mesmo que não seja o artigo mais citado, porque recebe influência de múltiplas áreas, aumentando sua importância global na rede.

---

Parte 5: Interpretação e Aplicações

Questão 13: Baseado nos resultados, o PageRank mede "importância", "popularidade" ou "autoridade" no contexto de uma enciclopédia? Justifique com exemplos específicos dos seus resultados.

Questão 14: Quais são as principais limitações do PageRank para este tipo de rede de conhecimento? Que fatores importantes sobre um artigo podem não ser capturados por esta métrica?

Questão 15: Como os resultados desta análise poderiam ser aplicados para melhorar a experiência de usuários navegando pela Wikipedia? Que recomendações você faria?

Questão 13: Baseado nos resultados, o PageRank mede "importância", "popularidade" ou "autoridade" no contexto de uma enciclopédia? Justifique com exemplos específicos dos seus resultados.

O PageRank mede a importância estrutural dos artigos na rede de links. Artigos como “United_States” e “France” são importantes por receber muitos links, enquanto artigos com muitos links de saída, como “Driving_on_the_left_or_right”, têm PageRank baixo. Artigos que conectam áreas diferentes, como “Currency”, também ganham relevância, mostrando que PageRank captura centralidade e influência na rede, não apenas popularidade.

Questão 14: Quais são as principais limitações do PageRank para este tipo de rede de conhecimento? Que fatores importantes sobre um artigo podem não ser capturados por esta métrica?

O PageRank apresenta limitações nesse tipo de rede de conhecimento porque considera apenas a estrutura de links entre os artigos, sem avaliar a qualidade ou a profundidade do conteúdo. Ele ignora o contexto semântico, ou seja, dois artigos podem estar ligados mesmo que não sejam relevantes entre si. Além disso, artigos com muitos links de saída podem ter seu score diluído, e conteúdos populares fora da rede ou recentes podem ser subestimados. Em suma, o PageRank mede centralidade estrutural, mas não reflete necessariamente autoridade, relevância ou importância informativa dos artigos.

Questão 15: Como os resultados desta análise poderiam ser aplicados para melhorar a experiência de usuários navegando pela Wikipedia? Que recomendações você faria?

Os resultados do PageRank podem ser usados para destacar artigos mais centrais ou influentes, ajudando usuários a encontrar conteúdos importantes rapidamente. Recomenda-se, por exemplo, sugerir links relacionados de alto PageRank nas páginas, criar rankings ou listas de artigos essenciais por área de conhecimento e otimizar a navegação entre tópicos conectados. Além disso, identificar “pontes” entre áreas permite oferecer percursos educativos interdisciplinares, conectando conceitos e facilitando a exploração de temas correlatos de forma mais intuitiva.

---

Dataset: WikiSpeedia - Snap Stanford University

SNAP: Web data: Wikispeedia navigation paths

Foco: Análise de Centralidade em Redes de Conhecimento